In [1]:
# 여러게시물 #상세내용 #수집 웹 크롤러 
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select






#1. 크롤링 기본 셋팅하기
print("=" *80)
print("연습문제 6-2 : 청와대 국민청원 게시판 크롤링하기 - 리스트_방식")
print("=" *80)
query_txt = '청와대국민청원-리스트'


cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 7)

no = 1

f_dir = input("2.결과 파일을 저장할 폴더명만 쓰세요(예:/Users/kibeomkim/Desktop/folder1/): ")
if f_dir=='' :
    f_dir='/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'

    
# 저장될 파일위치와 이름 지정하기
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name= s+'-'+query_txt+'.txt'
fc_name= s+'-'+query_txt+'.csv'
fx_name= s+'-'+query_txt+'.xls'


# 크롬 드라이버를 사용해서 웹 브라우저 실행하기 

s_time = time.time( )

path = "/Users/kibeomkim/Desktop/chromedriver_240/chromedriver"
driver = webdriver.Chrome(path)


a_url_list = []

#1페이지 
driver.get('https://www1.president.go.kr/petitions/?c=0&only=1&page=1&order=1')
time.sleep(random.randrange(2,5))

for a in range(1, page_cnt+1) : 

    #1 페이지 파싱
    html0 = driver.page_source 
    soup0 = BeautifulSoup(html0, 'html.parser')
    link_list_find1 = soup0.find('ul', 'petition_list').find_all('a', 'cb relpy_w')
    
    
    #1페이지에 있는 a 태그 url만 모아서 --> 리스트로 만들기
    for b in range(1, 8) : 
        b_1 = link_list_find1[b-1]
        real_url = b_1['href']
        a_url_list.append(real_url)
        no+=1
        if no > cnt : 
            break
    
    a+1 
    if a > page_cnt : 
        break
    #다음페이지로 넘어가기
    try : 
        driver.find_element_by_link_text('%s' %a).click()
    except : 
        driver.find_element_by_xpath('/html/body/div[2]/div[2]/section[2]/div[2]/div/div/div[2]/div[2]/div[4]/div/div[4]/div/div[2]/a').click()
    time.sleep(3)
# url 리스트 생성 완료 . 리스트 이름 : a_url_list 이다. 


pet_number_list = []

#청원 번호 리스트 생성하기 
for c in a_url_list : 
    pet_number = c.split('/')[2]
    pet_number_list.append(pet_number)

#청원 번호 리스트 생성완료. 리스트 이름 : pet_number_list 이다. 

#드라이버 닫기 
driver.close()


petition_title1 = []
parti_people1 = []
category1 = []
start_date = []
end_date = []
contents11 = []


import sys
original_stdout = sys.stdout 
f = open(ff_name, 'a', encoding='UTF-8')
sys.stdout = f


#청원url 리스트 이용해서 순서대로 내용 크롤링하기 
no1 = 1

for d in a_url_list : 

    print('%s 번째 국민청원 게시글 상세 정보입니다==============================' %no1)
    
    
    path = "/Users/kibeomkim/Desktop/chromedriver_240/chromedriver"
    driver = webdriver.Chrome(path)
    
    
    #해당 페이지로 간다
    driver.get('https://www1.president.go.kr'+d)
    time.sleep(3) #페이지 전환 대기 
    
    
    #청원번호 리스트 이용해서 txt파일에 입력하기 
    f.write('1. 청원번호 :'+pet_number_list[no1-1])
    f.write('\n')#//
    
    html1 = driver.page_source 
    soup1 = BeautifulSoup(html1, 'html.parser')
    #청원 제목 찾고 리스트 추가하기 
    pet_tit = soup1.find('h3','petitionsView_title').get_text()
    pet_tit01= pet_tit.replace('\n', "").strip()
    petition_title1.append(pet_tit01)#리스트 추가
    f.write('2. 청원제목 :'+pet_tit01)#txt 파일에 쓰기
    f.write('\n')#//
    
    #참여인원 찾고 리스트 추가하기 
    parti_p = soup1.find('h2', 'petitionsView_count').find('span', 'counter').get_text()
    parti_p01 = parti_p.replace('\n', "")
    parti_p02 = parti_p01.replace(',', "").strip()
    parti_people1.append(parti_p02)#리스트추가
    f.write('3. 참여인원 :'+parti_p02)#txt 파일에 쓰기
    f.write('\n')#//
    
    #카테고리 찾고 리스트 추가하기 
    text_list = soup1.find('ul', 'petitionsView_info_list').find_all('li')
    hangjungp = text_list[0].get_text()
    hangjungp1 = hangjungp.replace('\n', "").replace('카테고리', "").strip()
    category1.append(hangjungp1)#리스트추가
    f.write('4. 카테고리 :'+hangjungp1)
    f.write('\n')#//
    
    #청원시작일 찾고 리스트 추가하기 
    starts = soup1.find('ul', 'petitionsView_info_list').find_all('li')
    starts_d = starts[1].get_text()
    starts_d1 = starts_d.replace('\n', "").replace('청원시작', "").strip()
    start_date.append(starts_d1)#리스트추가
    f.write('5. 청원시작일 :'+starts_d1)
    f.write('\n')#//
    
    #청원종료일 찾고 리스트 추가하기 
    ends = soup1.find('ul', 'petitionsView_info_list').find_all('li')
    ends_d = ends[2].get_text()
    ends_d1 = ends_d.replace('\n', "").replace('청원마감', "").strip()
    end_date.append(ends_d1)#리스트추가
    f.write('6. 청원종료일 :'+ends_d1)
    f.write('\n')#//
    
    #청원내용 찾고 리스트 추가하기 
    contents = soup1.find('div', 'View_write').get_text()
    contents1 = contents.replace('\n', "").strip()
    contents11.append(contents1)#리스트추가
    f.write('7. 청원내용 :'+contents1)
    f.write('\n')#//
    
    
    f.write('\n')
    
    no1 +=1
    
    driver.close()
        
sys.stdout = original_stdout 
f.close()




    
#출력결과를 csv, xlsx로 저장하기

petition = pd.DataFrame()
petition['청원번호'] = pet_number_list
petition['청원제목'] = petition_title1
petition['참여자 수'] = parti_people1
petition['카테고리'] = category1
petition['청원시작일'] = start_date 
petition['청원종료일'] = end_date
petition['청원내용'] = contents11

#csv
petition.to_csv(fc_name, encoding='utf-8-sig', index=False)
#xlsx
petition.to_excel(fx_name, index=False)

e_time = time.time()
t_time = e_time - s_time # 검색 마무리하는데 까지 걸린 시간 계산


# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)


연습문제 6-2 : 청와대 국민청원 게시판 크롤링하기 - 리스트_방식
1.크롤링 할 건수는 몇건입니까?: 3
2.결과 파일을 저장할 폴더명만 쓰세요(예:/Users/kibeomkim/Desktop/folder1/): /Users/kibeomkim/Desktop/folder1


총 소요시간은 36.5 초 입니다 
파일 저장 완료: txt 파일명 : 2020-12-29-00-17-43-청와대국민청원-리스트.txt 
파일 저장 완료: csv 파일명 : 2020-12-29-00-17-43-청와대국민청원-리스트.csv 
파일 저장 완료: xls 파일명 : 2020-12-29-00-17-43-청와대국민청원-리스트.xls 
